# Reference systems

> **Goal**: Explain the reference systems used for crystal orientation in FTIR and EBSD

TODO

## FTIR reference system

TODO

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.spatial.transform import Rotation as r

TODO

## EBSD reference system

TODO

![EBSD_unit_cell_convention](https://raw.githubusercontent.com/bertopadron/FTIR-EBSD/main/notebooks2learn/imgs/EBSD_ref_frame.png)

TODO

![EBSD_ref_frame](https://raw.githubusercontent.com/bertopadron/FTIR-EBSD/main/notebooks2learn/imgs/EBSD_unit_cell_convention.png)

In [2]:
import sys
import scipy as sp
from datetime import date    
today = date.today().isoformat()

print(f'Notebook tested in {today} using:')
print('Python', sys.version)
print('Numpy', np.__version__)
print('Scipy', sp.__version__)

Notebook tested in 2023-02-08 using:
Python 3.10.9 | packaged by conda-forge | (main, Jan 11 2023, 15:15:40) [MSC v.1916 64 bit (AMD64)]
Numpy 1.23.5
Scipy 1.9.3
